## 装饰器

## （函数）装饰器（语法糖@）
- 函数装饰器指的是 装饰器 是一个函数

### 1. 被修饰的函数不带参数的装饰器

In [1]:
# 1. 被修饰的函数不带参数的装饰器， 即 func()没有参数
def use_logging(func):

    def wrapper():
        print("%s is running" % func.__name__)
        return func()   # 把 foo 当做参数传递进来时，执行func()就相当于执行foo()
    return wrapper      

@use_logging             # 等价于 foo = use_logging(foo)
def foo():
    print('i am foo')
    
foo()

foo is running
i am foo


### 2. 被修饰的函数带参数的装饰器 func(*args, **kwargs)

In [2]:
# 2. 被修饰的函数带参数的装饰器 ---> 即 func(...)有参数， 可以用动态参数：*args、**kwargs
def use_logging(func):

    def wrapper(*args, **kwargs):
        print("%s is running" % func.__name__)
        return func(*args, **kwargs)   # 把 foo 当做参数传递进来时，执行func()就相当于执行foo()
    return wrapper      

@use_logging             # 等价于 foo = use_logging(foo)
def foo(name, age=None, height=None):
    print("I am %s, age %s, height %s" % (name, age, height))
    
foo('tom', 20)

foo is running
I am tom, age 20, height None


###  3. 包裹函数带参数的修饰器 deco(param)

In [3]:
def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                print("WARN: %s is running" % func.__name__)
            elif level == "info":
                print("INFO: %s is running" % func.__name__)
            return func(*args)
        return wrapper

    return decorator

# @use_logging(level="warn")      # 包裹函数带参数指定level等级
@use_logging(level='info')
def foo(name='foo'):
    print("i am %s" % name)
    
foo()

INFO: foo is running
i am foo


## （类）装饰器
- 类装饰器指的是 装饰器 是一个函数

In [4]:
import time
from functools import wraps

class tracer:
    '''
    class decorator
    '''

    def __init__(self, func):
        self.call = 0
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print("Decorator '{}' was called.".format(self.__class__.__name__))
        self.call += 1
        print("Function '{}' was called {} times".format(self.func.__name__, self.call))
        # self.func(*args, **kwargs)
        return self.func(*args, **kwargs)
    
@tracer
def sum_ab(a, b):
    time.sleep(0.2210)
    return a ** 2 + a * b + b ** 2

print(sum_ab(2,3))
wraps

Decorator 'tracer' was called.
Function 'sum_ab' was called 1 times
19


<function functools.wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))>

### 装饰器修复技术 @wraps(func)
- 将 被修饰的函数(wrapped) 的一些属性值赋值给 修饰器函数(wrapper) ，最终让属性的显示更符合我们的直觉
- @wraps接受一个函数来进行装饰，并加入了复制函数名称、注释文档、参数列表等等的功能。这可以让我们在装饰器里面访问在装饰之前的函数的属性。
- 不改变使用装饰器原有函数的结构(如__name__, __doc__), 不使用wraps可能出现的ERROR:   view_func...endpoint...map...

In [5]:
# 装饰器
def logged(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        '''logging docstrings'''
        return func(*args, **kwargs)
    return with_logging

# 函数
@logged
def f(x):
   """does some math"""
   return x + x * x

res = f(2)
print('res: ', res)
print(f.__name__)     # 没有 @wraps, 输出 with_logging；      有, 输出 f
print(f.__doc__)      # 没有 @wraps, 输出 logging docstrings；有, 输出does some math

res:  6
f
does some math


In [6]:
def wrapper(f):
    def wrapper_function(*args, **kwargs):
        """这个是修饰函数"""
        return f(*args, **kwargs)
    return wrapper_function
    
@wrapper
def wrapped():
    """这个是被修饰的函数"""
    print('wrapped')
    
print(wrapped.__doc__)
print(wrapped.__name__)

这个是修饰函数
wrapper_function


- 如果`没使用@wraps`，当A调用了装饰器B的话，`A.__name__`返回的会是`装饰器B的函数名称`，而不是A的函数名称
- 如果`使用了@wraps`，当A调用了装饰器B的话，`A.__name__`返回的会是`A函数的名称`，而不是饰器B的名称.
这也即使常说的@wraps是装饰器的修复技术，实际就是修复还原了A的__name__变量，同理__doc__变量也是一样。

#### 装饰器顺序
一个函数还可以同时定义多个装饰器，比如：

它的执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器，它等效于